In [1]:
import nltk
from nltk.tokenize import sent_tokenize
from transformers import MarianMTModel, MarianTokenizer
from sentence_transformers import SentenceTransformer, util

In [2]:
# Download necessary NLTK data
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /home/szinjad/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/szinjad/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [3]:
def paraphrase_sentence(sentence):
    # Initialize models
    paraphrase_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

    # Generate embeddings for the sentence
    sentence_embedding = paraphrase_model.encode(sentence, convert_to_tensor=True)
    
    # Generate paraphrases
    paraphrases = util.semantic_search(sentence_embedding, [sentence_embedding], top_k=5)
    
    # Return the top paraphrase (excluding the original sentence)
    return paraphrases[0][1]['text']

In [4]:

def back_translate(sentence):
    # Initialize models
    translation_model_en_de = MarianMTModel.from_pretrained('Helsinki-NLP/opus-mt-en-de')
    translation_tokenizer_en_de = MarianTokenizer.from_pretrained('Helsinki-NLP/opus-mt-en-de')
    translation_model_de_en = MarianMTModel.from_pretrained('Helsinki-NLP/opus-mt-de-en')
    translation_tokenizer_de_en = MarianTokenizer.from_pretrained('Helsinki-NLP/opus-mt-de-en')
    
    # Translate to German
    de_tokens = translation_tokenizer_en_de(sentence, return_tensors="pt", padding=True)
    de_translation = translation_model_en_de.generate(**de_tokens)
    de_text = translation_tokenizer_en_de.decode(de_translation[0], skip_special_tokens=True)
    
    # Translate back to English
    en_tokens = translation_tokenizer_de_en(de_text, return_tensors="pt", padding=True)
    en_translation = translation_model_de_en.generate(**en_tokens)
    en_text = translation_tokenizer_de_en.decode(en_translation[0], skip_special_tokens=True)
    
    return en_text

In [5]:

def perturb_text(text):
    sentences = sent_tokenize(text)
    perturbed_sentences = []
    
    for sentence in sentences:
        # Paraphrase
        paraphrased = paraphrase_sentence(sentence)
        perturbed_sentences.append(("Paraphrase", paraphrased))
        
        # Back-translate
        back_translated = back_translate(sentence)
        perturbed_sentences.append(("Back-translation", back_translated))
    
    return perturbed_sentences

In [6]:

# Example usage
input_text = "The quick brown fox jumps over the lazy dog. It was a beautiful day outside."
perturbed_text = perturb_text(input_text)

for perturbation_type, perturbed_sentence in perturbed_text:
    print(f"{perturbation_type}: {perturbed_sentence}")

/home/szinjad/.conda/envs/llm_safety_39/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


IndexError: list index out of range